In [17]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import v2

# DATA EXPLORATION

In [22]:
# Directory structure:
    # Apple
    # ├── Apple_Black_rot
    # │   ├── Augmented
    # │   │   ├── image (1)_Blur.jpg
    # │   │   ├── image (1)_Flip.jpg
    # │   │   ├── ...
    # │   ├── image (1).jpg 
    # │   ├── image (2).jpg
    # │   ├── ...
    # ├── Apple_Healthy
    # │   ├── Augmented
    # │   │   ├── image (1)_Blur.jpg
    # │   │   ├── image (1)_Flip.jpg
    # │   │   ├── ...
    # │   ├── image (1).jpg
    # │   ├── image (2).jpg
    # │   ├── ...
    # ├── ...
    # Grape
    # ├── ...

def distribution(path: str):
    # Initialize a counter for the diseases
    disease_counter = {}

    # Traverse the directory
    for root, dirs, files in os.walk(path):
        # Skip the root directory itself
        if root == path:
            continue

        # Check if we are in a disease folder (not in the augmented subfolder)
        if 'Augmented' in root:
            continue

        disease_name = os.path.basename(root)
        if disease_name not in disease_counter:
            disease_counter[disease_name] = 0
    
        # Count the images in the current directory
        for file in files:
            if file.lower().endswith('.jpg'):
                disease_counter[disease_name] += 1
        # Count the images in the Augmented directory if exists
        augmented_path = os.path.join(root, 'Augmented')
        if os.path.exists(augmented_path):
            for file in os.listdir(augmented_path):
                if file.lower().endswith('.jpg'):
                    disease_counter[disease_name] += 1

    labels = list(disease_counter.keys())
    sizes = list(disease_counter.values())

    # Plot pie chart
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('Pie Chart')

    # Plot bar chart
    plt.subplot(1, 2, 2)
    plt.bar(labels, sizes, color='skyblue')
    plt.xlabel('Plant Disease')
    plt.ylabel('Number of Images')
    plt.title('Bar Chart')
    plt.xticks(rotation=45, ha='right')

    plt.tight_layout()
    plt.show()

In [ ]:
distribution('./images/Apple')

# DATA AUGMENTATION

In [35]:
def ImageAugmentation(src: str, img: str):
    image = Image.open(img)
    imgname = img.split('/')
    imgname = ('/').join(imgname[-1:])

    blur = v2.GaussianBlur(kernel_size=(3, 7),
                            sigma=(1.5, 6))(image)
    blur.save(os.path.join(src,
                            imgname + "_blur.jpg"))

    rotate = v2.RandomRotation(degrees=[-80, 80])(image)
    rotate.save(os.path.join(src,
                                imgname.split('.')[0] + "_rotate.jpg"))

    contrast = v2.ColorJitter(brightness=(0.5, 2),
                                contrast=(0.5, 2),
                                saturation=(0.5, 2))(image)
    contrast.save(os.path.join(src,
                                imgname.split('.')[0] +
                                "_contrast.jpg"))

    scaling = v2.RandomResizedCrop(size=(200, 200))(image)
    scaling.save(os.path.join(src,
                                imgname.split('.')[0] + "_scaling.jpg"))

    shear = v2.RandomAffine(degrees=0, shear=45)(image)
    shear.save(os.path.join(src,
                            imgname.split('.')[0] + '_shear.jpg'))

    if random.randint(0, 1) == 0:
        flip = v2.functional.hflip(image)
    else:
        flip = v2.functional.vflip(image)
    flip.save(os.path.join(src,
                            imgname.split('.')[0] + '_flip.jpg'))

def Augmentation(src: str):
    if src.lower().endswith('.jpg'):
        newsrc = src.split('/')
        newsrc = ('/').join(newsrc[:-1])
        os.makedirs(newsrc + '/Augmented', exist_ok=True)
        ImageAugmentation(newsrc + '/Augmented', src)
    else:
        os.makedirs(src + '/Augmented', exist_ok=True)
        for img in os.listdir(src):
            newsrc = os.path.join(src, img)
            if img.lower().endswith('.jpg'):
                ImageAugmentation(src + '/Augmented', newsrc)
            

In [36]:
path = "./images/Apple/Apple_rust/image (1).jpg"
path2 = "./images/Grape/Grape_healthy"
Augmentation(path)

# DATA TRANSFORMATION

In [38]:
from plantcv import plantcv as pcv

def gaussian_blur(mask):
    pcv.gaussian_blur(img=mask, ksize=(3, 3))


def apply_mask(image, mask):
    pcv.apply_mask(img=image, mask=mask, mask_color='white')


def rect_roi(image):
    pcv.roi.rectangle(img=image, x=35, y=8, h=235, w=200)


def shape_analysis(image, mask):
    pcv.analyze.size(img=image, labeled_mask=mask)


def x_pseudolandmarks(image, mask):
    pcv.homology.x_axis_pseudolandmarks(img=image, mask=mask)


def y_pseudolandmarks(image, mask):
    pcv.homology.y_axis_pseudolandmarks(img=image, mask=mask)


def Transformation(src: str, dest: str = None):
    if src.lower().endswith('.jpg'):
        pcv.params.debug = "plot"
        image, _, _ = pcv.readimage(filename=src)

        # Identify the grayscale image that maximizes
        # the gray value difference between the plant
        # and the background with:
        # cs = pcv.visualize.colorspaces(rgb_img=image,
        #                            original_img=False)
        # We deduce that the 'LAB' colorspace's channel
        # 'a' is the best option for our grayscale image
        grayscale_img = pcv.rgb2gray_lab(rgb_img=image, channel="a")

        # Identify plant from background, by converting
        # the grayscale image to a binary image (MASK)
        # using a threshold value of '119' which is
        # the minimum value between the two peaks in
        # our histogram:
        # hist = pcv.visualize.histogram(img=grayscale_img,
        #                               bins=30)
        s_thresh = pcv.threshold.binary(grayscale_img,
                                        threshold=119,
                                        object_type="dark")
        # Apply Transformations
        gaussian_blur(s_thresh)
        apply_mask(image, s_thresh)
        rect_roi(image)
        shape_analysis(image, s_thresh)
        x_pseudolandmarks(image, s_thresh)
        y_pseudolandmarks(image, s_thresh)
    else:
        pcv.params.debug = "print"
        if dest:
            pcv.params.debug_outdir = dest
            os.makedirs(f"{dest}", exist_ok=True)
        else:
            pcv.params.debug_outdir = f"{src}/Transformed"
            os.makedirs(f"{src}/Transformed", exist_ok=True)
        for img in os.listdir(src):
            if img.lower().endswith('.jpg'):
                image, _, _ = pcv.readimage(
                    filename=os.path.join(src, img))
                grayscale_img = pcv.rgb2gray_lab(rgb_img=image,
                                                 channel="a")
                s_thresh = pcv.threshold.binary(grayscale_img,
                                                threshold=119,
                                                object_type="dark")
                gaussian_blur(s_thresh)
                apply_mask(image, s_thresh)
                rect_roi(image)
                shape_analysis(image, s_thresh)
                x_pseudolandmarks(image, s_thresh)
                y_pseudolandmarks(image, s_thresh)

    pcv.outputs.save_results(filename="result.json",
                             outformat="json")

In [43]:
src = "C:\\Users\\khali\\Desktop\\Leaffliction\\images\\Apple\\Apple_healthy"
Transformation(src)